In [1]:
#Checking for GPU availability
!nvidia-smi

Sat Sep 21 11:47:31 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Step 1: Install Necessary Libraries


Step 2: Load and Prepare the Dataset
Step 3: Preprocess the Data
tep 4: Set Up the Training Arguments
Step 5: Initialize the Model
Step 6: Train the Model
Step 7: Evaluate the Model
Step 8: Save the Fine-Tuned Model
Predictions for new text

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 6.6 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0
Found existing installation: transformers 4.44.0
Uninstalling transformers-4.44.0:
  Successfully uninstalled transformers-4.44.0
Found existing installation: accelerate 0.34.2
Uninstalling accelerate-0.34.2:
  Successfully uninstalled accelerate-0.34.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
  Using cached accelerate-0.34.2-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 59.5 MB/s eta 0:00:0000:0100:01
Using cached accelerate-0.34.2-py3-none-any.whl (324 kB)


In [3]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

**AutoTokenizer.from_pretrained(model_ckpt):** initializes the tokenizer associated with the model checkpoint. The tokenizer is responsible for converting text data into the format that the PEGASUS model can understand (such as tokenizing the text into subword tokens). It uses the AutoTokenizer class, which automatically loads the appropriate tokenizer for the selected model checkpoint.

# Loading pre trained Model

In [6]:
# Choosing pre-trained Pegasus model for sequence-to-sequence tasks
# Below link shows you all summarization model available in Hugging face.
# Please try to use a different model and train.
# https://huggingface.co/models?pipeline_tag=summarization&sort=trending
model_ckpt = "google/pegasus-cnn_dailymail"
# Selecting the same model Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


**AutoModelForSeq2SeqLM.from_pretrained(model_ckpt):**
This function loads a pre-trained model from the Hugging Face Transformers library.

**AutoModelForSeq2SeqLM** is a class that automatically selects the correct model architecture for sequence-to-sequence tasks.

**model_ckpt** is the checkpoint path or model identifier, which specifies the exact Pegasus model to load (e.g., google/pegasus-large).

**.to(device)**:
This moves the model to the device you are using, such as a GPU ('cuda') or CPU ('cpu'). This ensures that all model operations are performed on the correct hardware.

If device = torch.device('cuda'), the model will be moved to a GPU.
If device = torch.device('cpu'), the model will be on a CPU.

In [7]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

# Loading DATASET

**Loading the SAMSum Dataset:**
The function load_dataset is part of the datasets library from Hugging Face, and it's used to load a predefined dataset. In this case, the dataset being loaded is the "SAMSum" dataset, which is designed for abstractive dialogue summarization.

**SAMSum Dataset:**
The SAMSum dataset consists of a large collection of dialogues written in natural language. It is typically used to train models on summarizing dialogues, such as chat conversations or short message exchanges, into concise summaries.

In [8]:
dataset_samsum = load_dataset("samsum")

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [9]:
# Dataset stats
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [10]:
# Printing a row of dialogue
dataset_samsum["train"]["dialogue"][20]


"Ashley: Guys, you have to read this book!  <file_photo>\r\nMarcus: Why, what's so special about it?\r\nErin: I think I've already heard about it from someone. Is it really that good?\r\nAshley: It's the best thing I've ever read! Completely life-changing! It's opened my eyes to a lot of things.\r\nSeamus: Sorry, but I don't like books that are written to change my life. I prefer books that are simply fun to read :P\r\nMarcus: I get what you mean. I feel like some authors are so concentrated on making their books full of wisdom that they completely forget that they should also be readable.\r\nErin: Do you mean Coelho? XD\r\nMarcus: No, while I'm not a fan of his, at least I've never fallen asleep while reading his books. I meant this one for example: <file_other>\r\nAshley: Erm, I quite like his books.\r\nSeamus: Did they change your life too? :D\r\nAshley: Wait, I meant Coelho. I've never read the other guy.\r\nMarcus: Trust me, don't. There are lots of better ways of wasting your tim

In [11]:
#printing a row of Summary
dataset_samsum["train"][20]["summary"]

"Erin is convinced by Ashley's book recommendations, while Seamus and Marcus aren't."

In [12]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


# Preprocessing

**tokenizer** is applied to the dialogue text, converting it into tokenized inputs (subword tokens and related data).

**max_length=1024** limits the input sequence to 1024 tokens. If the dialogue exceeds this length, the truncation=True argument ensures that the input is **truncated** to the maximum length.

The function returns a dictionary containing the following:

**input_ids:** These are the token IDs corresponding to the dialogue text (the input).

**attention_mask:** A mask that indicates which tokens are actual input tokens (1s) and which are padding (0s). This helps the model ignore the padding during training.

**labels:** These are the token IDs corresponding to the summary (the target). During training, the model tries to generate these target labels based on the input dialogue.

In [13]:
#preprocessing date and convert to vector representation.
#returns input Id, attention Mask and Labels
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [14]:
#Mapping the function(convert_examples_to_features) to entire dataset
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [15]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [16]:
dataset_samsum_pt["train"]["input_ids"][1]

[18038,
 151,
 2632,
 127,
 119,
 6228,
 118,
 115,
 136,
 2974,
 152,
 10463,
 151,
 35884,
 130,
 329,
 107,
 18038,
 151,
 2587,
 314,
 1242,
 10463,
 151,
 1509,
 1]

In [17]:
dataset_samsum_pt["train"]["attention_mask"][1]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

**DataCollatorForSeq2Seq:**

This is a special data collator provided by Hugging Face for sequence-to-sequence (seq2seq) tasks, such as text summarization or translation.
A data collator is responsible for dynamically batching the input examples during training, ensuring they are of the right format for the model, while also handling padding and other necessary preprocessing.
#### **What DataCollatorForSeq2Seq Does:**

**Dynamic Padding:** When input sequences in a batch have different lengths, they need to be padded to the length of the longest sequence in the batch. This collator adds the padding tokens where necessary to make all sequences the same length.

**Label Padding:** It also pads the labels (the target summaries) so they match the length of the longest label in the batch.
Handling Special Tokens: In seq2seq tasks, models often need specific tokens (like start tokens or end tokens) for the decoder. The data collator ensures that these tokens are properly added.

**Efficient Batching:** It allows efficient batching of sequences with varying lengths, avoiding the need to manually pad the data beforehand.

In [18]:
# Training
# DataCollatorForSeq2Seq helps to data to load in Batches 
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

# Fine Tuning Model

#### **TrainingArguments**
TrainingArguments is Hugging Face’s transformers library is used to define all the settings and configurations for the training process. Here's what each parameter in the code does.

**output_dir='pegasus-samsum':** 
This specifies the directory where the trained model, checkpoints, and logs will be saved. In this case, it will save the outputs in the directory 'pegasus-samsum'.

**num_train_epochs=1:**
This sets the number of times the model will pass through the entire training dataset. In this case, it is set to train for 1 epoch.

**warmup_steps=500:**
This is the number of warm-up steps before the learning rate begins to decay. During the first 500 steps, the learning rate will increase from 0 to the maximum value, which helps stabilize the training.

**per_device_train_batch_size=1:**
This sets the batch size for training on each device (e.g., GPU). A batch size of 1 means that only 1 training sample will be processed at a time per device.

**per_device_eval_batch_size=1:**
This sets the batch size for evaluation on each device. Similar to training, only 1 sample will be evaluated at a time.

**weight_decay=0.01:**
This applies L2 regularization to prevent overfitting by penalizing large weights during training. A weight decay of 0.01 applies a small penalty.

**logging_steps=10:**
This determines how often (in terms of steps) the model's training progress will be logged. Here, it will log the training progress every 10 steps.

**evaluation_strategy='steps':**
This defines how often the evaluation is performed. Setting it to 'steps' means the model will be evaluated at regular step intervals (as opposed to at the end of each epoch).

**eval_steps=500:**
This specifies that evaluation will be run every 500 training steps.

**save_steps=1e6:**
This defines how often (in terms of steps) the model's checkpoint will be saved. A value of 1e6 means that the model will not save checkpoints frequently (1 million steps is quite a large interval), likely to save space and time.

**gradient_accumulation_steps=16:**
This accumulates gradients over 16 steps before performing a backward pass. Effectively, this allows the model to simulate a larger batch size by accumulating gradients over several steps before updating the model's weights. It helps when you have a small per_device_train_batch_size.

In [19]:
trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


We creates a Trainer object, which orchestrates the training process for the model.

**model=model_pegasus:**
This specifies the model to be trained. In this case, model_pegasus refers to the pre-trained PEGASUS model (likely the "google/pegasus-cnn_dailymail" model used for summarization tasks).

**args=trainer_args:**
These are the training arguments that define how the training will proceed. This refers to the trainer_args object that was defined earlier using the TrainingArguments class. It includes settings like batch size, evaluation strategy, number of epochs, and logging steps.

**tokenizer=tokenizer:**
The tokenizer is responsible for converting the raw text into input tokens that the model can understand. The tokenizer here refers to the PEGASUS tokenizer initialized earlier, which tokenizes both the input dialogues and their corresponding summaries.

**data_collator=seq2seq_data_collator:**
The data collator (seq2seq_data_collator) dynamically pads input sequences and labels during training, ensuring that the input and output sequences in a batch have uniform lengths. This is especially important in sequence-to-sequence tasks like summarization where the inputs and targets can vary in length.

**train_dataset=dataset_samsum_pt["test"]:**
The training dataset is specified here. here we uses the "test" split of the SAMSum dataset for training. you would use the "train" split for training, but test set is being used, inorder to train this model fast and this is a experimental setup. I urge you to use train dataset.

**eval_dataset=dataset_samsum_pt["validation"]:**
The evaluation dataset is set to the "validation" split of the SAMSum dataset (dataset_samsum_pt["validation"]). This dataset is used to evaluate the model during training, providing feedback on how well the model is performing on unseen data.

In [20]:

trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])

In [21]:
#initiates the training process for the model with all the settings, 
#datasets, and arguments 
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ·····································


wandb: ERROR API key must be 40 characters long, yours was 37
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ·····································


wandb: ERROR API key must be 40 characters long, yours was 37
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


TrainOutput(global_step=25, training_loss=3.162765884399414, metrics={'train_runtime': 891.2668, 'train_samples_per_second': 0.919, 'train_steps_per_second': 0.028, 'total_flos': 423944250507264.0, 'train_loss': 3.162765884399414, 'epoch': 0.975609756097561})

# Evaluation

#### **generate_batch_sized_chunks**

This function splits a list of elements (in this case, articles or summaries) into smaller, batch-sized chunks to process in parallel.
This function is useful when you need to process large datasets but want to break them into manageable pieces, typically due to memory limitations.

#### **calculate_metric_on_test_ds**

This function performs the evaluation of the model on the test dataset, generating summaries for each batch and calculating a metric like ROUGE (used to evaluate text generation tasks like summarization).

**Key Parameters:**

**dataset:** The dataset containing the articles and their reference summaries.

**metric:** The evaluation metric, like ROUGE, BLEU, or others.

**model:** The trained model (e.g., PEGASUS) that generates summaries.

**tokenizer:** The tokenizer that processes the text into model-readable tokens and decodes the outputs.

**batch_size:** The number of samples to process in each batch (default is 16).
device: The computing device (e.g., GPU) to which tensors are moved for faster processing.

**column_text:** The column name in the dataset that contains the articles.
column_summary: The column name that contains the reference summaries.


**Batch Creation:**

The dataset is split into batches using the generate_batch_sized_chunks function. Both the articles and reference summaries are split into batches of size batch_size.

**Tokenization:**

Each batch of articles is tokenized with a maximum length of 1024 tokens and padded to ensure uniform length. The input tensors are prepared using return_tensors="pt" to create PyTorch tensors.

**Model Summarization:**

The model generates summaries for the tokenized inputs using beam search (num_beams=8), a technique that improves the quality of generated sequences. The length_penalty ensures the generated summaries are not too long or too short.

**Decoding:**

The generated summaries are decoded back into readable text using tokenizer.decode. The skip_special_tokens=True ensures that special tokens like [CLS] and [SEP] are not included in the decoded summaries.

**Metric Calculation:**

For each batch of generated summaries and reference summaries, the metric is updated using metric.add_batch(). This batch-wise processing helps reduce memory usage for large datasets.

**Final Metric Computation:**

After processing all batches, the final score (e.g., ROUGE score) is computed using metric.compute().

In [22]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


**load_metric('rouge'):** Loads the ROUGE metric, which will be used to evaluate the quality of generated text (summaries) by comparing it with the reference text (gold summaries).

The ROUGE (Recall-Oriented Understudy for Gisting Evaluation) metric is commonly used to assess the quality of automatically generated text, such as summaries, by measuring the overlap between the generated and reference texts.

**rouge1:** Measures the overlap of unigrams (single words) between the generated summary and the reference summary.

**rouge2:** Measures the overlap of bigrams (two consecutive words) between the generated and reference summaries.

**rougeL:** Measures the longest common subsequence (LCS) between the generated and reference summaries. This metric captures the fluency and structure of the generated text.

**rougeLsum:** Similar to rougeL, but it specifically focuses on summarization tasks and measures the LCS across the entire summary.

In [23]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

/tmp/ipykernel_36/2696170338.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

<ipython-input-25-5a43aadd1b0e>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate

  rouge_metric = load_metric('rouge')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py

You can avoid this message in future by passing the argument `trust_remote_code=True`.

Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.

  warnings.warn(


In [24]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

100%|██████████| 5/5 [00:15<00:00,  3.12s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.01559,0.0,0.015116,0.015444


In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

100%|██████████| 5/5 [00:10<00:00,  2.13s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.017267,0.0,0.017017,0.017225


# Saving Model

In [25]:
## Save model to folder
model_pegasus.save_pretrained("pegasus-samsum-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [26]:
## Save tokenizer to folder
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

# Prediction

In [29]:
#Load the tokenizer

tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/tokenizer")

In [ ]:
#Prediction with fine tuned model

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:

Hannah: Hey, do you have Betty's number?

Amanda: Lemme check

Hannah: <file_gif>

Amanda: Sorry, can't find it.

Amanda: Ask Larry

Amanda: He called her last time we were at the park together

Hannah: I don't know him well

Hannah: <file_gif>

Amanda: Don't be shy, he's very nice

Hannah: If you say so..

Hannah: I'd rather you texted him

Amanda: Just text him 🙂

Hannah: Urgh.. Alright

Hannah: Bye

Amanda: Bye bye



Reference Summary:

Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.



Model Summary:

Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .


In [31]:
#Prediction with fine tuned model

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset_samsum["test"][20]["dialogue"]

reference = dataset_samsum["test"][20]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Dialogue:
Deirdre: Hi Beth, how are you love?
Beth: Hi Auntie Deirdre, I'm been meaning to message you, had a favour to ask.
Deirdre: Wondered if you had any thought about your Mum's 40th, we've got to do something special!
Beth: How about a girls weekend, just mum, me, you and the girls, Kira will have to come back from Uni, of course.
Deirdre: Sounds fab! Get your thinking cap on, it's only in 6 weeks! Bet she's dreading it, I remember doing that!
Beth: Oh yeah, we had a surprise party for you, you nearly had a heart attack! 
Deirdre: Well, it was a lovely surprise! Gosh, thats nearly 4 years ago now, time flies! What was the favour, darling?
Beth: Oh, it was just that I fancied trying a bit of work experience in the salon, auntie.
Deirdre: Well, I am looking for Saturday girls, are you sure about it? you could do well in the exams and go on to college or 6th form.
Beth: I know, but it's not for me, auntie, I am doing all foundation papers and I'm struggling with those.
Deirdre: What

###  Explanation of the Model's Performance

**What Went Well:**

**Core Theme Identified:**

The model successfully identified a key part of the conversation, which is Beth wanting to work at Deidre's salon. This is central to the dialogue and is captured well in the model’s summary.
The intention of Beth trying out work at the salon is correctly conveyed. This shows that the model is able to pick up on the most important aspect of the dialogue and generate a coherent and relevant summary around it.

**What Went Wrong:**

**Missed Details:**

The model fails to mention Beth's plans to organize a girls' weekend for her mother’s 40th birthday. This is a significant part of the reference summary, and its omission suggests the model struggles to capture multiple key details when they are spread across the dialogue.

**Incomplete Work Experience Context:**

The model only partially captures the work experience aspect. It simplifies the interaction between Deidre and Beth by not mentioning that Beth is only offered a few hours on Saturdays. This loss of specificity can impact the clarity and depth of the summary.

**Limited Vocabulary and Simplification:**

The model's language is simpler and more generic compared to the reference. For instance, phrases like "try working in a salon" and "beauty therapy side" simplify the actual context, which affects the richness of the summary.
It did not fully utilize the opportunity to describe the work experience context in a nuanced way. The model generated a more straightforward version, which might indicate an issue with understanding subtle differences in dialogue.

**Potential Issues with the Model:**

**Truncation or Loss of Information:**

Since the reference contains multiple key points, it's possible the model was truncated or did not focus on all aspects of the dialogue, prioritizing only one central theme (the salon work). You may want to check whether the model’s input/output lengths are too restricted.

**Beam Search Parameters:**

The model is using beam search (num_beams=8), but increasing the beam size or adjusting the length penalty could improve the ability to capture more complex or detailed summaries.

**Training Data Bias:**

If the training data primarily focused on shorter or simpler dialogues, the model might generalize by producing summaries that lack depth or details in more complex conversations.

**Improvements to Consider:**

**Fine-tuning the Model Further**: You could fine-tune the model on dialogues that emphasize multi-faceted conversations, so it learns to capture all important details.

**Length Penalty Adjustment:** Increasing the length penalty could encourage the model to generate slightly longer and more complete summaries that capture more details.

**Adjusting Training Data:** Including more examples where multiple key points need to be captured (like plans, intentions, and actions) may help the model generalize better to this type of scenario.

Please consider improving the model with all above mentioned techniques for a fruitfull hands on.